In [129]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys,os
from textblob import TextBlob,  Word
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import Text
import random
from wordcloud import WordCloud
from nltk import ngrams
from imageio import imread
from collections import Counter
#%run datuslib.ipynb

#ROBADO DE LAS NOTE DE CLASES
plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

In [130]:
#CARGO DATASET

M_tweets = pd.read_csv('train.csv')
M_tweets.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [131]:
def extractWordAfterSymbol(text,symbol):
    
    """ 
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: Una lista de todos los strings que estaban precedidos por un 'symbol'
    
    
    Ejemplo User: symbol='@'
    EJEMPLO: 'Hola que tal soy @axel, y vivo en @capitalFederal' -----> [axel,capitalFederal]'
    
    Ejemplo hashtag: symbol='#'
    Ejemplo: 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all' --->[earthquake]
    """
    
    words = text.split(' ')
    
    users = []
    
    for word in words:
        userToSave = ""
        if len(word) != 0 and symbol in word:
            splittedWord = word.split(symbol)
            if len(splittedWord) > 1 and (splittedWord[0]=='' or splittedWord[0]==' ' ):
                userToSave = splittedWord[1]
        if userToSave != '':
            users.append(userToSave)
    return users

In [132]:
def takeOutSpecialCaractersFromText(text, specialCharacters = None):
    '''
    RECIBE: Un string.
    DEVUELVE: El string sin caracteres especiales. Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    if specialCharacters == None:
        specialCharacters = ['&','\n','<','>','=',',','.','!','%','#','*','(',')','"', '?', '$', ':', '/','\'']
    
    
    for char in specialCharacters:
        text = text.replace(char, '')
    return text

In [133]:
def textWithoutStopwords(text):
    specialCharacters = ['&','\n','<','>','=',',','.','!','%','*','(',')','"','-','_', '?', '$', ':', '/','\''] + [str(i) for i in range(0,10)]
    output=""
    for word in text.split():
        if ((word not in stop_words) | (word not in specialCharacters) ):
            output+=word
            output+=" "
    output=output[:-1]
    return output

In [134]:
def stemmedText(text):
    ps = PorterStemmer()
    toProcess=text.split()
    result=""
    for word in toProcess:
        result+=ps.stem(word)+" "
    result=result[:-1]    
    return result

In [135]:
def list2text(list):
    text=""
    for w in list:
        text+=w
        text+=" "
    text=text[:-1]    
    return text

In [136]:
def filterMoreThanTwoChar(text):   
    result=""
    for word in text.split():
        if (len(word)>1):
            result+=word
            result+=" "
    result=result[:-1]  
    return result

46.	Ver propiedades estadísticas de los tweets que mencionan otros usuarios
47.	Usar a los usuarios arrobados para ver cosas de ellos


In [156]:
M_tweets['textNoStop']=M_tweets['text'].apply(lambda x: textWithoutStopwords(x))
M_tweets['textNoStop2']=M_tweets['textNoStop'].apply(lambda x:takeOutSpecialCaractersFromText(x))
M_tweets['textNoStop3']=M_tweets['textNoStop2'].apply(lambda x:filterMoreThanTwoChar(x))
M_tweets['qWords']=M_tweets['textNoStop'].apply(lambda x: len(x.split()))
M_tweets.head()

,id,keyword,location,text,target,textNoStop,textNoStop2,textNoStop3,users,hashtag,hasUsers,hasHashtag,qWords
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this earthquake Ma...,Our Deeds are the Reason of this earthquake Ma...,[],[earthquake],0,1,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask Canada,Forest fire near La Ronge Sask Canada,[],[],0,0,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,All residents asked to shelter in place are be...,All residents asked to shelter in place are be...,[],[],0,0,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",13000 people receive wildfires evacuation orde...,13000 people receive wildfires evacuation orde...,[],[wildfires],0,1,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,Just got sent this photo from Ruby Alaska as s...,Just got sent this photo from Ruby Alaska as s...,[],"[alaska, wildfires]",0,2,16


In [157]:
M_tweets['users']=M_tweets['textNoStop3'].apply(lambda x: extractWordAfterSymbol( x.lower(),'@'))
M_tweets['hashtag']=M_tweets['textNoStop'].apply(lambda x: extractWordAfterSymbol( x.lower(),'#'))
M_tweets.head()

,id,keyword,location,text,target,textNoStop,textNoStop2,textNoStop3,users,hashtag,hasUsers,hasHashtag,qWords
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this earthquake Ma...,Our Deeds are the Reason of this earthquake Ma...,[],[earthquake],0,1,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask Canada,Forest fire near La Ronge Sask Canada,[],[],0,0,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,All residents asked to shelter in place are be...,All residents asked to shelter in place are be...,[],[],0,0,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",13000 people receive wildfires evacuation orde...,13000 people receive wildfires evacuation orde...,[],[wildfires],0,1,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,Just got sent this photo from Ruby Alaska as s...,Just got sent this photo from Ruby Alaska as s...,[],"[alaska, wildfires]",0,2,16


In [158]:
M_tweets['hasUsers']=M_tweets['users'].apply(lambda x: len(x))
M_tweets['hasHashtag']=M_tweets['hashtag'].apply(lambda x: len(x))
M_tweets.head(50)

,id,keyword,location,text,target,textNoStop,textNoStop2,textNoStop3,users,hashtag,hasUsers,hasHashtag,qWords
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this earthquake Ma...,Our Deeds are the Reason of this earthquake Ma...,[],[earthquake],0,1,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask Canada,Forest fire near La Ronge Sask Canada,[],[],0,0,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,All residents asked to shelter in place are be...,All residents asked to shelter in place are be...,[],[],0,0,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...",13000 people receive wildfires evacuation orde...,13000 people receive wildfires evacuation orde...,[],[wildfires],0,1,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,Just got sent this photo from Ruby Alaska as s...,Just got sent this photo from Ruby Alaska as s...,[],"[alaska, wildfires]",0,2,16
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,#RockyFire Update => California Hwy. 20 closed...,RockyFire Update California Hwy 20 closed in ...,RockyFire Update California Hwy 20 closed in b...,[],"[rockyfire, cafire, wildfires]",0,3,18
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,#flood #disaster Heavy rain causes flash flood...,flood disaster Heavy rain causes flash floodin...,flood disaster Heavy rain causes flash floodin...,[],"[flood, disaster]",0,2,14
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,I'm on top of the hill and I can see a fire in...,Im on top of the hill and I can see a fire in ...,Im on top of the hill and can see fire in the ...,[],[],0,0,15
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,There's an emergency evacuation happening now ...,Theres an emergency evacuation happening now i...,Theres an emergency evacuation happening now i...,[],[],0,0,12
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,I'm afraid that the tornado is coming to our a...,Im afraid that the tornado is coming to our area,Im afraid that the tornado is coming to our area,[],[],0,0,10


In [159]:
M_tweetsUsers=M_tweets[~(M_tweets['hasUsers']==0)].copy()
M_tweetsUsers.head()

,id,keyword,location,text,target,textNoStop,textNoStop2,textNoStop3,users,hashtag,hasUsers,hasHashtag,qWords
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,@bbcmtd Wholesale Markets ablaze http://t.co/l...,@bbcmtd Wholesale Markets ablaze httptcolHYXEO...,@bbcmtd Wholesale Markets ablaze httptcolHYXEO...,[bbcmtd],[],1,0,5
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0,@PhDSquares #mufc they've built so much hype a...,@PhDSquares mufc theyve built so much hype aro...,@PhDSquares mufc theyve built so much hype aro...,[phdsquares],[mufc],1,1,21
43,63,ablaze,NaN,SOOOO PUMPED FOR ABLAZE ???? @southridgelife,0,SOOOO PUMPED FOR ABLAZE ???? @southridgelife,SOOOO PUMPED FOR ABLAZE @southridgelife,SOOOO PUMPED FOR ABLAZE @southridgelife,[southridgelife],[],1,0,6
54,78,ablaze,Abuja,Noches El-Bestia '@Alexis_Sanchez: happy to se...,0,Noches El-Bestia '@Alexis_Sanchez: happy to se...,Noches El-Bestia @Alexis_Sanchez happy to see ...,Noches El-Bestia @Alexis_Sanchez happy to see ...,[alexis_sanchez],[],1,0,15
57,81,ablaze,"Sao Paulo, Brazil",Set our hearts ablaze and every city was a gif...,0,Set our hearts ablaze and every city was a gif...,Set our hearts ablaze and every city was a gif...,Set our hearts ablaze and every city was gift ...,[û_],[],1,0,22


# USUARIOS MAS FRECUENTES

In [160]:
freqUsers = FreqDist(takeOutSpecialCaractersFromText(list2text(M_tweetsUsers.users.sum())).split())
commonUsers=freqUsers.most_common()
top15MostUsers=commonUsers[:15]
top15MostUsers

[('youtube', 83),
 ('arianagrande', 11),
 ('potus', 9),
 ('change', 9),
 ('usatoday', 9),
 ('foxnews', 9),
 ('emmerdale', 8),
 ('djicemoon', 7),
 ('justinbieber', 7),
 ('mikeparractor', 6),
 ('stretcher', 6),
 ('towel', 6),
 ('viralspell', 5),
 ('armageddon', 5),
 ('usagov', 5)]

# USUARIOS MAS FRECUENTES EN DESASTRES

In [161]:
M_tweetsUsersDisaster=M_tweetsUsers[M_tweetsUsers['target']==1]
freqWordsDisaster = FreqDist(takeOutSpecialCaractersFromText(list2text(M_tweetsUsersDisaster.users.sum())).split())
commonWordsDisaster=freqWordsDisaster.most_common()
top15MostCommonDisaster=commonWordsDisaster[:15]
top15MostCommonDisaster

[('youtube', 18),
 ('foxnews', 8),
 ('potus', 6),
 ('usagov', 5),
 ('change', 4),
 ('ap', 4),
 ('usatoday', 4),
 ('unsuckdcmetro', 4),
 ('viralspell', 3),
 ('nbcnews', 3),
 ('michael5sos', 3),
 ('kurtschlichter', 3),
 ('sharethis', 3),
 ('reuters', 3),
 ('smh', 3)]

# USUARIOS MAS FRECUENTES EN NO DESASTRES DESASTRES

In [162]:
M_tweetsUsersNoDisaster=M_tweetsUsers[M_tweetsUsers['target']==0]
freqWordsNoDisaster = FreqDist(takeOutSpecialCaractersFromText(list2text(M_tweetsUsersNoDisaster.users.sum())).split())
commonWordsNoDisaster=freqWordsNoDisaster.most_common()
top15MostCommonNoDisaster=commonWordsNoDisaster[:15]
top15MostCommonNoDisaster

[('youtube', 65),
 ('arianagrande', 11),
 ('djicemoon', 7),
 ('emmerdale', 7),
 ('justinbieber', 7),
 ('mikeparractor', 6),
 ('change', 5),
 ('usatoday', 5),
 ('stretcher', 5),
 ('invalid', 5),
 ('towel', 5),
 ('armageddon', 4),
 ('local_arsonist', 4),
 ('lonewolffur', 4),
 ('gop', 4)]

In [163]:
M_tweetsUsersDisaster[M_tweetsUsersDisaster.users.apply(lambda x: 'usatoday' in x)].iloc[0,:].text

'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/HQsU8LWltH via @usatoday'

In [164]:
M_tweetsUsersNoDisaster[M_tweetsUsersNoDisaster.users.apply(lambda x: 'invalid' in x)].iloc[0,:].text

'@Stretcher @invalid @Grazed @Rexyy @Towel I see the programme (:'

# Alguna metricas

In [165]:
M_tweetsUsers.groupby('target')['hasUsers'].agg(['max','mean','min','size'])

,max,mean,min,size
target,,,,
0,8,1.348021,1,1339
1,5,1.330793,1,656


# Cantidad de hashtag por cantidad de usuarios @

In [166]:
M_tweetsUsers.groupby('hasUsers').hasHashtag.agg(['sum','mean']).reset_index()

,hasUsers,sum,mean
0,1,467,0.306029
1,2,102,0.301775
2,3,42,0.518519
3,4,1,0.037037
4,5,13,0.812500
5,6,3,0.600000
6,7,1,1.000000
7,8,1,1.000000


# Cantidad de palabras de los tweets ordenados por cantidad de users @

In [168]:
M_tweetsUsers.groupby('hasUsers').qWords.agg(['sum','mean']).reset_index()

,hasUsers,sum,mean
0,1,23190,15.196592
1,2,5238,15.497041
2,3,1338,16.518519
3,4,448,16.592593
4,5,241,15.062500
5,6,61,12.200000
6,7,15,15.000000
7,8,11,11.000000
